In [ ]:
%pip install -e ../../../pipeline
%pip install -e ../../../../data-inclusion-schema


In [ ]:
import os

import dotenv
import pandas as pd

from data_inclusion.scripts.tasks import grist
from data_inclusion import schema

In [ ]:
dotenv.load_dotenv(dotenv.find_dotenv())

GRIST_API_TOKEN = os.environ["GRIST_API_TOKEN"]
GRIST_API_URL = "https://grist.incubateur.net/api"
WORKSPACE_ID = "27"
DOCUMENT_NAME = "template"

In [ ]:
grist_client = grist.GristClient(base_url=GRIST_API_URL, token=GRIST_API_TOKEN)

document_id = grist_client.create_document(
    workspace_id=WORKSPACE_ID, document_name=DOCUMENT_NAME
)

In [ ]:
for referentiel in [
    "frais",
    "labels_nationaux",
    "modes_accueil",
    "modes_orientation_accompagnateur",
    "modes_orientation_beneficiaire",
    "profils",
    "thematiques",
    "typologies_de_services",
    "typologies_de_structures",
    "zones_de_diffusion_types",
]:
    table_id = grist_client.create_table(
        document_id=document_id,
        table_name=referentiel.capitalize(),
        columns=[
            {"id": "value", "fields": {"label": "valeur", "type": "Text"}},
            {"id": "label", "fields": {"label": "label", "type": "Text"}},
        ],
    )

    referentiel_df = pd.read_csv(
        f"../../../pipeline/dbt/seeds/schema/{referentiel}.csv",
        dtype=str,
    )

    # attention: pas idempotent

    grist_client.add_records(
        document_id=document_id,
        table_id=table_id,
        records=[
            {"fields": value_dict}
            for value_dict in referentiel_df[["value", "label"]].to_dict(
                orient="records"
            )
        ],
    )

In [ ]:
import re


def get_column_type(field) -> str:
    match_referentiel = re.search(
        r"data_inclusion.schema.(?P<referentiel>\w+)", str(field.annotation)
    )

    if match_referentiel is not None:
        return "Ref:" + match_referentiel.group("referentiel").capitalize()
    elif "float" in str(field.annotation):
        return "Numeric"
    elif "bool" in str(field.annotation):
        return "Bool"
    elif "date" in str(field.annotation):
        return "DateTime:Europe/Paris"

    return "Text"

In [ ]:
grist_columns = [
    {
        "id": field_name,
        "fields": {
            "label": field_name,
            "type": get_column_type(field_info),
            # "visibleCol": TODO
        },
    }
    for field_name, field_info in schema.Structure.model_fields.items()
]

grist_client.create_table(
    document_id=document_id,
    table_name="Structures",
    columns=grist_columns,
)

In [ ]:
grist_columns = [
    {
        "id": field_name,
        "fields": {
            "label": field_name,
            "type": get_column_type(field_info),
            # "visibleCol": TODO
        },
    }
    for field_name, field_info in schema.Service.model_fields.items()
]

grist_client.create_table(
    document_id=document_id,
    table_name="Services",
    columns=grist_columns,
)
